In [1]:
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime, timedelta
import gspread
from gspread_dataframe import set_with_dataframe
import time

In [2]:
# Symbols to process
symbols = ["XAUUSD", "BTCUSD", "BTCEUR", "ETHUSD", "SOLUSD", "XRPUSD", 
           "DSHUSD", "XAUEUR", "GBPAUD", "GBPNZD", "CHFJPY", 
           "EURJPY", "USA30", "USA100", "USOIL", "UKOIL", "USDJPY", 
           "GBPJPY", "USA500", "UK100", "CADJPY", "AUDJPY", "GBPCHF", 
           "GBPCAD", "NZDJPY", "GBPUSD", "EURGBP"]

# Initialize MetaTrader 5
if not mt5.initialize():
    print(f"MetaTrader 5 initialization failed with error code {mt5.last_error()}")
    exit()

# Check account information
account_info = mt5.account_info()
if account_info is None:
    print("Failed to connect to the trading account")
    mt5.shutdown()
    exit()

print(f"Connected to account: {account_info.login}, Server: {account_info.server}")

# Function to find the last available trading day
def get_last_trading_day(reference_date):
    """
    Finds the most recent trading day that has data available.
    """
    while True:
        if reference_date.weekday() in [5, 6]:  # Skip Saturday (5) and Sunday (6)
            reference_date -= timedelta(days=1)
        else:
            # Try fetching data for this date
            start = datetime(reference_date.year, reference_date.month, reference_date.day, 0, 0)
            end = start + timedelta(days=1)
            for symbol in symbols:
                rates = mt5.copy_rates_range(symbol, mt5.TIMEFRAME_D1, start, end)
                if rates is not None and len(rates) > 0:
                    return reference_date  # Found a valid trading day
            reference_date -= timedelta(days=1)  # Try the previous day if no data found

# Function to fetch trading data
def fetch_trading_data(symbols):
    """
    Fetch high, low, and level data for the given symbols over the past three days.
    Handles weekends by using the most recent available data.
    """
    results = []
    now = datetime.now()

    # Define labels and their respective reference days
    days = [("C", now), ("Y", now - timedelta(days=1)), ("V", now - timedelta(days=2))]

    for symbol in symbols:
        if not mt5.symbol_select(symbol, True):
            print(f"Symbol {symbol} is not available.")
            continue

        for level, date in days:
            # Check if data is missing due to weekends and find last available trading day
            if date.weekday() in [5, 6]:  # Saturday or Sunday
                date = get_last_trading_day(date)
            
            # Set the start and end date correctly
            start = datetime(date.year, date.month, date.day, 0, 0)  # Start of the day
            end = start + timedelta(days=1)  # End of the day

            print(f"Fetching data for {symbol} from {start} to {end}")

            # Fetch rates for the symbol
            rates = mt5.copy_rates_range(symbol, mt5.TIMEFRAME_D1, start, end)
            
            if rates is not None and len(rates) > 0:
                # Extract high and low values
                high = max(rate['high'] for rate in rates)
                low = min(rate['low'] for rate in rates)
                results.append({"Date": start.date(), "Symbol": symbol, "Highest": high, "Lowest": low, "Level": level})
            else:
                print(f"No data found for {symbol} on {start.date()}, using previous data")
                last_trading_day = get_last_trading_day(date)
                last_start = datetime(last_trading_day.year, last_trading_day.month, last_trading_day.day, 0, 0)
                last_end = last_start + timedelta(days=1)
                rates = mt5.copy_rates_range(symbol, mt5.TIMEFRAME_D1, last_start, last_end)

                if rates is not None and len(rates) > 0:
                    high = max(rate['high'] for rate in rates)
                    low = min(rate['low'] for rate in rates)
                    results.append({"Date": last_start.date(), "Symbol": symbol, "Highest": high, "Lowest": low, "Level": level})
                else:
                    results.append({"Date": last_start.date(), "Symbol": symbol, "Highest": None, "Lowest": None, "Level": level})

    return pd.DataFrame(results)

# Function to upload data to Google Sheets
def upload_to_google_sheets(data, json_path, sheet_name):
    """
    Save the data to Google Sheets.
    """
    try:
        # Authenticate with Google Sheets
        gc = gspread.service_account(filename=json_path)
        sh = gc.open(sheet_name)
        worksheet = sh.sheet1

        # Clear old data and write new data
        worksheet.clear()
        set_with_dataframe(worksheet, data)

        print(f"Data successfully uploaded to Google Sheets: {sh.url}")
    except Exception as e:
        print(f"Error uploading data to Google Sheets: {e}")

# Main function with continuous updates
def main():
    """
    Main function to fetch trading data and upload it to Google Sheets at regular intervals.
    """
    json_path = "C:/Users/Shaheera/Downloads/live-data-449002-016cfa9e813e.json"  # Replace with your path
    sheet_name = "live_data_updated"  # Replace with your Google Sheets name

    try:
        print("Starting continuous data fetch and upload (Press Ctrl+C to stop)...")
        while True:
            # Fetch data
            trading_data = fetch_trading_data(symbols)

            # Upload to Google Sheets
            upload_to_google_sheets(trading_data, json_path, sheet_name)

            # Wait for 1 minute (adjustable interval)
            time.sleep(60)
    except KeyboardInterrupt:
        print("\nProcess stopped by user.")
    except Exception as e:
        print(f"Error during execution: {e}")
    finally:
        # Shutdown MetaTrader 5 connection
        mt5.shutdown()

Connected to account: 48690778, Server: HFMarketsGlobal-Demo


In [ ]:
if __name__ == "__main__":
    main()

Starting continuous data fetch and upload (Press Ctrl+C to stop)...
Fetching data for XAUUSD from 2025-01-31 00:00:00 to 2025-02-01 00:00:00
Fetching data for XAUUSD from 2025-01-30 00:00:00 to 2025-01-31 00:00:00
Fetching data for XAUUSD from 2025-01-29 00:00:00 to 2025-01-30 00:00:00
Fetching data for BTCUSD from 2025-01-31 00:00:00 to 2025-02-01 00:00:00
Fetching data for BTCUSD from 2025-01-30 00:00:00 to 2025-01-31 00:00:00
Fetching data for BTCUSD from 2025-01-29 00:00:00 to 2025-01-30 00:00:00
Fetching data for BTCEUR from 2025-01-31 00:00:00 to 2025-02-01 00:00:00
Fetching data for BTCEUR from 2025-01-30 00:00:00 to 2025-01-31 00:00:00
Fetching data for BTCEUR from 2025-01-29 00:00:00 to 2025-01-30 00:00:00
Fetching data for ETHUSD from 2025-01-31 00:00:00 to 2025-02-01 00:00:00
Fetching data for ETHUSD from 2025-01-30 00:00:00 to 2025-01-31 00:00:00
Fetching data for ETHUSD from 2025-01-29 00:00:00 to 2025-01-30 00:00:00
Symbol SOLUSD is not available.
Fetching data for XRPUSD